<a href="https://colab.research.google.com/github/machiwao/CCTHESS1-CCTHESS2-Dev-and-Docs/blob/eco/EDITED_ERA5_Land_Daily_Aggregated_'14_'23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api --upgrade
!pip install geemap --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.2/463.2 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 1.5.24
    Uninstalling earthengine-api-1.5.24:
      Successfully uninstalled earthengine-api-1.5.24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.5/631.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: geemap
    Found existing installation: geemap 0.35.3
    Uninstalling geemap-0.35.3:
      Successfully uninstalled geemap-0.35.3


In [14]:
import ee, pandas as pd
import geemap

In [15]:
ee.Authenticate()
ee.Initialize(project='midyear-forest-427007-s7')

In [16]:
# ERA5-Land Daily Aggregates dataset
dataset = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
  .filterDate('2014-01-01', '2023-12-31')

In [17]:
# Print available variables (bands)
bands = dataset.first().bandNames().getInfo()
print("Available Bands:", bands)
print("Dataset size:", dataset.size().getInfo())

Available Bands: ['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4', 'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature', 'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor', 'lake_total_layer_temperature', 'snow_albedo', 'snow_cover', 'snow_density', 'snow_depth', 'snow_depth_water_equivalent', 'snowfall_sum', 'snowmelt_sum', 'temperature_of_snow_layer', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'forecast_albedo', 'surface_latent_heat_flux_sum', 'surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum', 'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum', 'surface_thermal_radiation_downwards_sum', 'evaporation_from_bare_soil_sum', 'evaporation_from_open_water_surfaces_excludi

In [18]:
# FIXED: Corrected coordinates for Aparri, Philippines
aparri = ee.Geometry.Point([120.967866, 14.58841]).buffer(25000)  # [longitude, latitude]
print("Geometry created:", aparri.getInfo())

def extract(img):
    # Add error handling and debugging
    vals = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aparri,
        scale=11132,  # ERA5-Land native resolution is ~11km
        maxPixels=1e13,
        bestEffort=True  # Add this to handle large geometries
    )

    # Get the date
    date_val = img.date().format("YYYY-MM-dd")

    # Create feature with both values and date
    feature = ee.Feature(None, vals).set("date", date_val)
    return feature

# Apply extraction to all images
fc = dataset.map(extract)

Geometry created: {'type': 'Polygon', 'coordinates': [[[120.96786599999999, 14.813381648787036], [120.90287096234647, 14.80441875684961], [120.8430645121824, 14.778245647483182], [120.79321748686297, 14.736951386211501], [120.757299998146, 14.683830443324032], [120.73816473367778, 14.623118103205073], [120.73732193104519, 14.55965119059787], [120.75482325095537, 14.498481792850583], [120.78926248141731, 14.444475107448755], [120.83789151249255, 14.401923406768075], [120.89684112320015, 14.374206448620322], [120.9614284163124, 14.363524718248645], [121.02652662963362, 14.370726044267391], [121.0869687965079, 14.39523885683813], [121.13795446620601, 14.43511717840958], [121.1754284919592, 14.487193916660425], [121.19640275628721, 14.547330739319815], [121.19919557701331, 14.610745316411142], [121.1835692977282, 14.672390552832288], [121.15075398570906, 14.72735608582676], [121.1033538655045, 14.77126018774863], [121.04514254894706, 14.800600570953595], [120.98076252595756, 14.81303554069

In [19]:
# Debug: Check if feature collection is properly created
print("Feature collection size:", fc.size().getInfo())

# Get a sample to check if data extraction worked
sample = fc.limit(5).getInfo()
print("Sample features:", sample['features'][0]['properties'])

Feature collection size: 3651
Sample features: {'date': '2014-01-01', 'dewpoint_temperature_2m': 293.30349763781817, 'dewpoint_temperature_2m_max': 294.6721996450529, 'dewpoint_temperature_2m_min': 292.2716390715004, 'evaporation_from_bare_soil_max': -1.766407455237496e-06, 'evaporation_from_bare_soil_min': -0.0003945315984503323, 'evaporation_from_bare_soil_sum': -0.002633826734979224, 'evaporation_from_open_water_surfaces_excluding_oceans_max': -1.3639029962469148e-06, 'evaporation_from_open_water_surfaces_excluding_oceans_min': -7.951805444134123e-05, 'evaporation_from_open_water_surfaces_excluding_oceans_sum': -0.0005969085335879532, 'evaporation_from_the_top_of_canopy_max': 2.999470577416962e-06, 'evaporation_from_the_top_of_canopy_min': -3.0494254636506304e-05, 'evaporation_from_the_top_of_canopy_sum': -3.0296667587941317e-05, 'evaporation_from_vegetation_transpiration_max': -7.287366983889875e-06, 'evaporation_from_vegetation_transpiration_min': -3.357115818421967e-05, 'evaporat

In [20]:
# Extract dates and data
dates = fc.aggregate_array("date").getInfo()
print(f"Number of dates extracted: {len(dates)}")
print(f"First few dates: {dates[:5]}")

var_names = dataset.first().bandNames().getInfo()
print(f"Variable names: {var_names}")

Number of dates extracted: 3651
First few dates: ['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04', '2014-01-05']
Variable names: ['dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature', 'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4', 'lake_bottom_temperature', 'lake_ice_depth', 'lake_ice_temperature', 'lake_mix_layer_depth', 'lake_mix_layer_temperature', 'lake_shape_factor', 'lake_total_layer_temperature', 'snow_albedo', 'snow_cover', 'snow_density', 'snow_depth', 'snow_depth_water_equivalent', 'snowfall_sum', 'snowmelt_sum', 'temperature_of_snow_layer', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 'forecast_albedo', 'surface_latent_heat_flux_sum', 'surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum', 'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum', 'su

In [21]:
# FIXED: Better error handling for data extraction
data = {}
for v in var_names:
    try:
        values = fc.aggregate_array(v).getInfo()
        data[v] = values
        print(f"Extracted {len([x for x in values if x is not None])} non-null values for {v}")
    except Exception as e:
        print(f"Error extracting {v}: {e}")
        data[v] = [None] * len(dates)

# Create DataFrame
df = pd.DataFrame({"date": dates})

Extracted 3651 non-null values for dewpoint_temperature_2m
Extracted 3651 non-null values for temperature_2m
Extracted 3651 non-null values for skin_temperature
Extracted 3651 non-null values for soil_temperature_level_1
Extracted 3651 non-null values for soil_temperature_level_2
Extracted 3651 non-null values for soil_temperature_level_3
Extracted 3651 non-null values for soil_temperature_level_4
Extracted 3651 non-null values for lake_bottom_temperature
Extracted 3651 non-null values for lake_ice_depth
Extracted 3651 non-null values for lake_ice_temperature
Extracted 3651 non-null values for lake_mix_layer_depth
Extracted 3651 non-null values for lake_mix_layer_temperature
Extracted 3651 non-null values for lake_shape_factor
Extracted 3651 non-null values for lake_total_layer_temperature
Extracted 3651 non-null values for snow_albedo
Extracted 3651 non-null values for snow_cover
Extracted 3651 non-null values for snow_density
Extracted 3651 non-null values for snow_depth
Extracted 36

In [22]:
# Add data columns with null checking
for v in var_names:
    if v in data:
        df[v] = data[v]
        null_count = df[v].isnull().sum()
        print(f"Column {v}: {len(df[v]) - null_count} valid values, {null_count} null values")

print("\nDataFrame shape:", df.shape)
print("DataFrame info:")
print(df.info())
print("\nFirst few rows:")
print(df.head())

Column dewpoint_temperature_2m: 3651 valid values, 0 null values
Column temperature_2m: 3651 valid values, 0 null values
Column skin_temperature: 3651 valid values, 0 null values
Column soil_temperature_level_1: 3651 valid values, 0 null values
Column soil_temperature_level_2: 3651 valid values, 0 null values
Column soil_temperature_level_3: 3651 valid values, 0 null values
Column soil_temperature_level_4: 3651 valid values, 0 null values
Column lake_bottom_temperature: 3651 valid values, 0 null values
Column lake_ice_depth: 3651 valid values, 0 null values
Column lake_ice_temperature: 3651 valid values, 0 null values
Column lake_mix_layer_depth: 3651 valid values, 0 null values
Column lake_mix_layer_temperature: 3651 valid values, 0 null values
Column lake_shape_factor: 3651 valid values, 0 null values
Column lake_total_layer_temperature: 3651 valid values, 0 null values
Column snow_albedo: 3651 valid values, 0 null values
Column snow_cover: 3651 valid values, 0 null values
Column sno

/tmp/ipython-input-19088604.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ipython-input-19088604.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ipython-input-19088604.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[v] = data[v]
/tmp/ipython-

In [23]:
# Unit conversions (only if data exists)
metadata = []
for col in df.columns:
    if col == "date":
        continue

    # Skip if all values are null
    if df[col].isnull().all():
        print(f"Skipping {col} - all values are null")
        continue

    original_name = col
    original_unit = "varies"
    converted_unit = original_unit
    new_name = col

    # Kelvin → Celsius
    if "temperature" in col.lower():
        df[col] = df[col] - 273.15
        new_name = col + "_C"
        original_unit, converted_unit = "K", "°C"
    # Precipitation / Runoff / Evaporation (m → mm/day)
    elif col.endswith("_sum") or "precipitation" in col.lower() or "evaporation" in col.lower() or "runoff" in col.lower():
        df[col] = df[col] * 1000
        new_name = col + "_mm"
        original_unit, converted_unit = "m", "mm/day"
    # Pressure (Pa → hPa)
    elif "pressure" in col.lower():
        df[col] = df[col] / 100
        new_name = col + "_hPa"
        original_unit, converted_unit = "Pa", "hPa"
    # Wind (m/s → km/h)
    elif "u_component" in col.lower() or "v_component" in col.lower() or "wind" in col.lower():
        df[col] = df[col] * 3.6
        new_name = col + "_kmh"
        original_unit, converted_unit = "m/s", "km/h"
    # Snow depth (m → cm)
    elif "snow_depth" in col.lower():
        df[col] = df[col] * 100
        new_name = col + "_cm"
        original_unit, converted_unit = "m", "cm"
    # Radiation & Fluxes (J/m²/day → W/m²)
    elif col.endswith("_radiation_sum") or col.endswith("_flux_sum") or col.endswith("_heat_sum"):
        df[col] = df[col] / 86400
        new_name = col + "_Wm2"
        original_unit, converted_unit = "J/m²/day", "W/m²"

    # Rename column
    if new_name != col:
        df.rename(columns={col: new_name}, inplace=True)

    # Add metadata record
    metadata.append({
        "Band": original_name,
        "Converted Column": new_name,
        "Original Unit": original_unit,
        "Converted Unit": converted_unit
    })

In [24]:
# Build metadata DataFrame
meta_df = pd.DataFrame(metadata)
print("\nMetadata:")
print(meta_df)


Metadata:
                                    Band                     Converted Column  \
0                dewpoint_temperature_2m            dewpoint_temperature_2m_C   
1                         temperature_2m                     temperature_2m_C   
2                       skin_temperature                   skin_temperature_C   
3               soil_temperature_level_1           soil_temperature_level_1_C   
4               soil_temperature_level_2           soil_temperature_level_2_C   
..                                   ...                                  ...   
145              total_precipitation_max           total_precipitation_max_mm   
146  leaf_area_index_high_vegetation_min  leaf_area_index_high_vegetation_min   
147  leaf_area_index_high_vegetation_max  leaf_area_index_high_vegetation_max   
148   leaf_area_index_low_vegetation_min   leaf_area_index_low_vegetation_min   
149   leaf_area_index_low_vegetation_max   leaf_area_index_low_vegetation_max   

    Original Uni

In [25]:
# Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# FIXED: Updated filename to reflect correct location (Aparri)
filename = "ERA5-Land_Daily_Aggregated_2014_2023_PortArea.csv"
drive_path = f'/content/drive/MyDrive/ERA5Datasets/{filename}'

try:
    df.to_csv(drive_path, index=False)
    print(f"File saved to Google Drive: {filename}")
    print(f"Final DataFrame shape: {df.shape}")
except Exception as e:
    print(f"Error saving file: {e}")
    # Save locally as backup
    df.to_csv(filename, index=False)
    print(f"File saved locally: {filename}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File saved to Google Drive: ERA5-Land_Daily_Aggregated_2014_2023_PortArea.csv
Final DataFrame shape: (3651, 151)
